A system of three linear equations has the following expression in matrix notation:
$$
\mathbf{y} = \mathbf{W}\mathbf{x}
$$

with
* $\mathbf{x}$ is a column vector representing our inputs whose size is (3,1).
* $\mathbf{y}$ is a column vector representing our outputs whose size is (3,1).
* $\mathbf{W}$ is a matrix representing our coefficients whose size is (3,3).

# Experiment
Let $X = \begin{bmatrix} 9. \\ 15. \\ 35. \end{bmatrix}$ and $W = \begin{bmatrix} 8. & 3. & -2. \\ -4. & 7. & 5. \\ 3. & 4. & -12. \end{bmatrix}$.

In [5]:
import torch
import torch.nn as nn
import numpy as np
import memtorch

X = [9., 15., 35.]
W = [[8., 3., -2.], [-4., 7., 5.], [3., 4., -12.]]

## Approach 1
We construct a single perceptron deep learning model with only a linear activation. No backward propagation is performed.

Once the model is implemented and mapped to the corresponding memristive deep neural network (MDNN), we feed the input vector $X$ to the corresponding memristive network.

In [6]:
class Reference(nn.Module):
    def __init__(self):
        super(Reference, self).__init__()
        pass

    def forward(self, X, W):
        self.X = torch.tensor(X, requires_grad= True)
        self.W = torch.tensor(W, requires_grad= True)
        result = torch.matmul(self.W, self.X)
        return result
    
##
# Define a reference model:
#
reference_model = Reference()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(in_features = 3, out_features = 3, bias = False)

    def forward(self, inputs):
      input_tensor = torch.tensor(inputs, requires_grad = True)
      output_tensor = self.fc(input_tensor)
      return output_tensor
    
##
# Define a test model:
#
test_model = Net()
with torch.no_grad():
  test_model.fc.weight.data = torch.tensor(W, requires_grad= True)

In [7]:
test_results = test_model.forward(X)
reference_results = reference_model.forward(X, W)


## Approach 2
We construct a memristor crossbar array. 

In [8]:
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Crossbar import init_crossbar

class IdealMCA():
    def __init__(self, X, W):
        super(IdealMCA, self).__init__()
        self.X = torch.tensor(X, requires_grad= True)
        self.W = torch.tensor(W, requires_grad= True)
        pass
    
    def define_memristor_behaviours(self):
        self.reference_memristor = memtorch.bh.memristor.VTEAM
        self.reference_memristor_params = {"time_series_resolution": 1e-10, "r_off": 100e3, "r_on": 10e3,}
        pass

    def define_crossbar(self):
        self.crossbars, self.operation = init_crossbar(
            weights = self.W,
            memristor_model= self.reference_memristor,
            memristor_model_params= self.reference_memristor_params,
            transistor= True,
            mapping_routine= naive_map,
            programming_routine= None,
            tile_shape= None,
            scheme= memtorch.bh.crossbar.Scheme.DoubleColumn,
        )
    
    def forward(self):
        self.define_memristor_behaviours()
        self.define_crossbar()

        self.W = self.operation(self.crossbars, lambda crossbar: crossbar.conductance_matrix)
        self.results = torch.matmul(self.W, self.X)
        print(f'Crossbar Product: {self.results}')
        return self.results 
        pass

test_model = IdealMCA(X, W)
test_results = test_model.forward()

Crossbar Product: tensor([ 0.0009,  0.0020, -0.0027], grad_fn=<MvBackward0>)


The reason this produces a scaled output is that the elements of $W$ are encoded on the crossbar using conductance values which can be represented by each device, i.e., they are scaled and encoded between $1/r_{off}$ and $1/r_{on}$. In MCA, the Ohm's law is exploited to perform Vector-Matrix Multiplication, so:
* Elements of tensor $X$ are encoded as Word Line (WL) voltages.
* Elements of tensor $Y$ is given as: $I=GV$ (assuming linear I/V characteristics and that line and source resistances are negligible/select devices are used). Since encoded elements of tensor $X$ are scaled, the resulting currents, i.e., $I$, must be scaled much that they are approximately equal to the desired product, which is $GV \equiv WX$.
